<a href="https://colab.research.google.com/github/ol3gka/AI_Masrters_NLA_project_1_Matrix_Masters/blob/main/Maxvol_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# maxvol-2 (rect-maxvol)
Автор: Николаев О.В.




**Algorithm 1 rect_maxvol (“Greedy” maximization of the volume of submatrix)**


**Require:** A full rank $A$




<div>
<img src="https://raw.githubusercontent.com/ol3gka/AI_Masrters_NLA_project_1_Matrix_Masters/main/pictures/max_vol_1.jpg" width="600"/>
</div>